### Домашнее задание, урок 2

In [27]:
import pandas as pd

Наши новости

In [28]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [29]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [30]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [31]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [32]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [33]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [34]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [35]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

Wall time: 53.7 s


In [36]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 7min 52s


А теперь в 3 строчки обучим нашу модель

In [37]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [38]:
common_dictionary[10]

'ватутин'

Все просто - это словарь наших слов

Запускаем обучение

In [39]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 1min 34s


In [40]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [41]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(1, 0.22323294), (13, 0.19223884), (16, 0.41284558), (18, 0.15297699)]

In [42]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: украина это который украинский свой год россия
topic_1: это который год человек эксперимент мочь весь
topic_2: журнал напомнить торговый зарплата завод зарегистрировать солнце
topic_3: миссия вино грузия гагарин недостаточный счесть перевод
topic_4: год млн который рост стать это банк
topic_5: гражданин снижение космос народный саммит донбасс армения
topic_6: ребёнок исследование смерть мозг пациент жизнь врач
topic_7: станция проект москва год центр площадь мероприятие
topic_8: район город китай остров житель иран китайский
topic_9: погибнуть nn вирус мышь пилот автомобиль результат
topic_10: год земля компания тыс который день это
topic_11: ii грунт орден небо приближаться воспринимать экспериментальный
topic_12: тело источник область обнаружить женщина суд дело
topic_13: это который свой мочь весь nn говорить
topic_14: край компьютер диск сахар зампред оперативно отслеживать
topic_15: кость млн выходной финляндия рим кит латвия
topic_16: рак двигатель озеро лётчик лёд австр

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [43]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))
    print("not_null_topics", not_null_topics)
    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [44]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

not_null_topics {0: 0.25000975, 9: 0.05936757, 16: 0.046184834, 20: 0.03911993, 24: 0.59797174}
not_null_topics {1: 0.6859919, 16: 0.24458893, 19: 0.04740295}
not_null_topics {1: 0.22327425, 13: 0.19219822, 16: 0.41284472, 18: 0.15297717}
not_null_topics {0: 0.0679062, 1: 0.24347764, 2: 0.02057581, 13: 0.43538874, 16: 0.22296137}
not_null_topics {4: 0.15270711, 14: 0.059563965, 16: 0.34284994, 24: 0.4214745}
not_null_topics {13: 0.66518617, 16: 0.3212622}
not_null_topics {0: 0.097998366, 8: 0.1173746, 9: 0.1562137, 16: 0.6001195}
not_null_topics {9: 0.67032045, 16: 0.29811457}
not_null_topics {1: 0.30094352, 4: 0.12111941, 9: 0.042764593, 15: 0.018928044, 16: 0.35427958, 24: 0.14906102}
not_null_topics {0: 0.034988716, 1: 0.13521707, 13: 0.52527326, 16: 0.28038234, 19: 0.012856634}
not_null_topics {4: 0.0793763, 9: 0.07865836, 15: 0.37150282, 16: 0.24590902, 20: 0.1985209}
not_null_topics {9: 0.64469135, 16: 0.08837901, 18: 0.06323035, 23: 0.16713713}
not_null_topics {1: 0.34222943, 9:

not_null_topics {0: 0.17307393, 4: 0.1369333, 9: 0.09392951, 13: 0.1930697, 20: 0.37862542}
not_null_topics {2: 0.214808, 13: 0.32411122, 16: 0.31381306, 22: 0.13951784}
not_null_topics {13: 0.37353078, 15: 0.038714234, 16: 0.1630178, 17: 0.012417072, 20: 0.28415692, 22: 0.11833504}
not_null_topics {4: 0.10036134, 14: 0.1616029, 15: 0.43363476, 20: 0.2754167}
not_null_topics {0: 0.22955622, 1: 0.08145438, 5: 0.017834628, 9: 0.039948586, 12: 0.3221667, 13: 0.08904656, 16: 0.029010693, 18: 0.027801486, 20: 0.15611883}
not_null_topics {2: 0.20049846, 4: 0.030720048, 9: 0.02129486, 13: 0.36675653, 16: 0.29899544, 24: 0.07239385}
not_null_topics {2: 0.3702307, 4: 0.10082904, 7: 0.10196146, 13: 0.25401774, 16: 0.15661608}
not_null_topics {5: 0.20556313, 16: 0.12674524, 22: 0.64847845}
not_null_topics {1: 0.13087252, 9: 0.2209608, 10: 0.24243358, 16: 0.39099833}
not_null_topics {2: 0.07150683, 4: 0.0881643, 10: 0.06164147, 13: 0.5290808, 14: 0.017228093, 16: 0.10246133, 20: 0.047868166, 22: 0

not_null_topics {0: 0.06608882, 14: 0.40541327, 15: 0.25092727, 16: 0.24572621}
not_null_topics {0: 0.13874504, 1: 0.041704062, 3: 0.041969888, 9: 0.4864533, 14: 0.10060916, 16: 0.14888835, 17: 0.026963133}
not_null_topics {5: 0.32357422, 16: 0.5752582, 18: 0.03134463, 19: 0.04431038}
not_null_topics {1: 0.12043592, 4: 0.14161623, 9: 0.6116717, 10: 0.0733906, 22: 0.032277253}
not_null_topics {1: 0.29834017, 8: 0.20492435, 9: 0.42989075, 19: 0.04055328}
not_null_topics {5: 0.15756106, 7: 0.059681352, 9: 0.2176997, 15: 0.05515267, 16: 0.4888236}
not_null_topics {12: 0.026988288, 13: 0.19854099, 16: 0.5466078, 20: 0.06442596, 24: 0.1473064}
not_null_topics {7: 0.044673417, 13: 0.14376676, 14: 0.028112119, 16: 0.1774379, 20: 0.38804823, 24: 0.20739801}
not_null_topics {4: 0.21797682, 16: 0.18651226, 20: 0.5213731, 21: 0.04511489}
not_null_topics {1: 0.19369195, 5: 0.18057014, 13: 0.22107925, 15: 0.13143654, 16: 0.25386292}
not_null_topics {0: 0.16754179, 4: 0.33307588, 10: 0.017582119, 13:

not_null_topics {1: 0.60237575, 5: 0.0249616, 13: 0.18509147, 16: 0.092466734, 20: 0.07656682, 23: 0.010933185}
not_null_topics {4: 0.30319992, 9: 0.100334324, 15: 0.3642261, 16: 0.0560062, 20: 0.1604922}
not_null_topics {1: 0.08141191, 8: 0.032428343, 13: 0.58810145, 16: 0.28579298}
not_null_topics {0: 0.07622065, 13: 0.11187632, 15: 0.21511419, 16: 0.18047854, 18: 0.16235964, 20: 0.24464448}
not_null_topics {3: 0.044431165, 4: 0.22581582, 15: 0.69580716}
not_null_topics {2: 0.0396881, 13: 0.78278255, 15: 0.09633765, 16: 0.069344446}
not_null_topics {0: 0.21572927, 2: 0.32813472, 13: 0.39231133, 16: 0.05355278}
not_null_topics {13: 0.41695324, 15: 0.034062617, 20: 0.52911866}
not_null_topics {5: 0.43024135, 6: 0.12683056, 16: 0.377737}
not_null_topics {4: 0.4637869, 15: 0.38558277, 16: 0.118411295}
not_null_topics {4: 0.17151242, 9: 0.09627101, 10: 0.05300089, 11: 0.046557438, 12: 0.06485761, 14: 0.21288994, 16: 0.22754198, 18: 0.113164015}
not_null_topics {4: 0.13390657, 13: 0.516540

not_null_topics {7: 0.40180716, 8: 0.25068355, 13: 0.23303793, 16: 0.10112282}
not_null_topics {4: 0.071138516, 12: 0.090553224, 13: 0.3948458, 15: 0.15899546, 16: 0.04380465, 20: 0.2195291}
not_null_topics {4: 0.20630708, 9: 0.21130574, 16: 0.21364987, 18: 0.08036353, 20: 0.25832564}
not_null_topics {4: 0.3159388, 15: 0.104857564, 16: 0.40935767, 24: 0.14983319}
not_null_topics {4: 0.34577978, 10: 0.038388405, 16: 0.5075787, 18: 0.04353596, 19: 0.029258206, 21: 0.018801568}
not_null_topics {1: 0.23518144, 4: 0.07836398, 7: 0.10769578, 10: 0.029675007, 14: 0.022225548, 16: 0.3237727, 18: 0.04009485, 21: 0.07048615, 24: 0.08591322}
not_null_topics {0: 0.034473065, 1: 0.29105636, 13: 0.362172, 16: 0.30107296}
not_null_topics {13: 0.4896964, 16: 0.09084834, 20: 0.18667173, 24: 0.21832341}
not_null_topics {1: 0.34709087, 16: 0.24426198, 24: 0.3946662}
not_null_topics {9: 0.6080954, 10: 0.36729193}
not_null_topics {0: 0.5023172, 1: 0.044476733, 7: 0.14975442, 13: 0.16088197, 16: 0.027263992

...

not_null_topics {4: 0.16589706, 6: 0.102827825, 11: 0.027575828, 13: 0.4004372, 20: 0.155207, 22: 0.1285507}
not_null_topics {4: 0.49391246, 6: 0.091670774, 13: 0.18005237, 20: 0.20890705}
not_null_topics {1: 0.20869188, 2: 0.109854475, 4: 0.01797949, 8: 0.027540758, 10: 0.035274275, 12: 0.0883124, 13: 0.30982274, 20: 0.04122103, 24: 0.15356925}
not_null_topics {0: 0.06346356, 1: 0.083936684, 9: 0.018565096, 13: 0.18401113, 24: 0.6369026}
not_null_topics {1: 0.6066525, 4: 0.12486946, 7: 0.10901154, 20: 0.115625724, 21: 0.02278237}
not_null_topics {1: 0.21909635, 4: 0.0872728, 7: 0.07695633, 10: 0.17141317, 13: 0.0857082, 20: 0.34302643}
not_null_topics {1: 0.30400103, 4: 0.15100563, 6: 0.121424675, 13: 0.17141463, 20: 0.23163702}
not_null_topics {0: 0.036162734, 1: 0.15881126, 4: 0.18526845, 7: 0.071565084, 9: 0.02506084, 22: 0.027002873, 24: 0.476254}
not_null_topics {1: 0.21343443, 6: 0.112030715, 11: 0.08632914, 13: 0.2812888, 20: 0.28585753}
not_null_topics {0: 0.31104136, 1: 0.070

not_null_topics {1: 0.12560867, 4: 0.1694459, 13: 0.2603472, 16: 0.032933377, 20: 0.29741347, 22: 0.09370406}
not_null_topics {1: 0.2507371, 4: 0.13540022, 13: 0.2359401, 16: 0.035373677, 19: 0.1015574, 20: 0.22331223}
not_null_topics {1: 0.121220134, 4: 0.116360754, 10: 0.10284777, 13: 0.23289318, 20: 0.1832249, 23: 0.02489046, 24: 0.21037842}
not_null_topics {1: 0.10910044, 9: 0.13798828, 13: 0.30771735, 20: 0.068423025, 21: 0.024465363, 22: 0.3328095}
not_null_topics {0: 0.071884684, 13: 0.08050265, 22: 0.41773006, 24: 0.42224142}
not_null_topics {10: 0.2003276, 13: 0.6443235, 20: 0.13277489}
not_null_topics {1: 0.09703222, 8: 0.20900394, 12: 0.118995614, 20: 0.36695725, 21: 0.07116492, 22: 0.11995274}
not_null_topics {0: 0.4038305, 2: 0.034728732, 8: 0.027354138, 15: 0.030691769, 22: 0.16477044, 24: 0.3259455}
not_null_topics {1: 0.2808181, 4: 0.17218047, 12: 0.13342853, 13: 0.10142302, 17: 0.028158879, 22: 0.2634462}
not_null_topics {4: 0.39973113, 5: 0.07458901, 13: 0.20923206, 2

not_null_topics {1: 0.2837905, 2: 0.039412383, 6: 0.027914153, 10: 0.12540671, 12: 0.027030451, 13: 0.31962204, 17: 0.023051003, 23: 0.024436766, 24: 0.11293953}
not_null_topics {1: 0.01691715, 9: 0.03990125, 10: 0.11597076, 13: 0.5189301, 20: 0.13387907, 24: 0.153279}
not_null_topics {1: 0.17737268, 6: 0.15420213, 12: 0.14624512, 13: 0.10681572, 14: 0.035221618, 17: 0.022209644, 22: 0.30086845, 24: 0.049505822}
not_null_topics {1: 0.186679, 6: 0.081319116, 9: 0.028015532, 12: 0.2104171, 13: 0.24215826, 20: 0.23952666}
not_null_topics {2: 0.0677304, 6: 0.010582608, 10: 0.11988821, 13: 0.683048, 20: 0.080089755, 21: 0.016750488}
not_null_topics {1: 0.05481525, 10: 0.07648391, 12: 0.024223514, 22: 0.18769489, 24: 0.6465208}
not_null_topics {0: 0.102658734, 12: 0.1166108, 13: 0.4163564, 18: 0.05123299, 20: 0.099124886, 22: 0.2065609}
not_null_topics {1: 0.29089007, 6: 0.08125971, 7: 0.100221746, 15: 0.072018825, 22: 0.2935892, 24: 0.14434049}
not_null_topics {1: 0.14476049, 4: 0.30042866,

not_null_topics {2: 0.043544024, 4: 0.21196833, 6: 0.19249818, 13: 0.19214162, 16: 0.03973744, 24: 0.29476336}
not_null_topics {1: 0.077176094, 4: 0.085112356, 7: 0.25587413, 10: 0.26384237, 12: 0.2548413, 24: 0.045474663}
not_null_topics {1: 0.21570013, 7: 0.053844426, 10: 0.23952588, 21: 0.026315555, 22: 0.21190624, 24: 0.2332128}
not_null_topics {5: 0.02913428, 10: 0.5717186, 13: 0.29372516, 24: 0.0844165}
not_null_topics {4: 0.33541742, 6: 0.096766904, 12: 0.23590927, 14: 0.10013069, 20: 0.14256573, 22: 0.0723161}
not_null_topics {4: 0.094037704, 8: 0.12607588, 10: 0.2780698, 22: 0.23146804, 24: 0.24609227}
not_null_topics {1: 0.07067486, 4: 0.37884057, 12: 0.15720752, 13: 0.19648197, 14: 0.026228413, 16: 0.05327271, 22: 0.09972638}
not_null_topics {0: 0.06888313, 1: 0.1683375, 4: 0.39218965, 7: 0.20573045, 8: 0.13285413}
not_null_topics {1: 0.0773251, 4: 0.2579548, 8: 0.065322354, 11: 0.017182805, 12: 0.044503298, 13: 0.14798108, 14: 0.022679681, 16: 0.07171439, 20: 0.14785135, 22

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.250010,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.046185,0.0,0.000000,0.000000,0.03912,0.0,0.0,0.0,0.597972
1,4896,0.000000,0.685992,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.244589,0.0,0.000000,0.047403,0.00000,0.0,0.0,0.0,0.000000
2,4897,0.000000,0.223274,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.412845,0.0,0.152977,0.000000,0.00000,0.0,0.0,0.0,0.000000
3,4898,0.067906,0.243478,0.020576,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.222961,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000
4,4899,0.000000,0.000000,0.000000,0.0,0.152707,0.0,0.0,0.0,0.0,...,0.0,0.342850,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.421474


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [46]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [47]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [48]:
doc_dict[293622]

array([0.        , 0.        , 0.        , 0.        , 0.10791034,
       0.        , 0.06544209, 0.        , 0.        , 0.        ,
       0.        , 0.01630765, 0.03211772, 0.31070837, 0.        ,
       0.        , 0.        , 0.        , 0.02855318, 0.01587502,
       0.        , 0.        , 0.1986025 , 0.        , 0.21476121])

In [49]:
user_articles_list = users['articles'].iloc[33]

Модифицируем функцию get_user_embedding_mean, чтобы она могла использовать не только среднее (mean), но также медиану и максимум.

In [50]:
def get_user_embedding(user_articles_list, stat_type):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if stat_type == 'mean':
        user_vector = np.mean(user_vector, 0)
    elif stat_type == 'median':
        user_vector = np.median(user_vector, 0)
    elif stat_type == 'max':
        user_vector = np.max(user_vector, 0)
    return user_vector

In [51]:
get_user_embedding(user_articles_list, 'mean')

array([0.12992022, 0.02416871, 0.00259987, 0.        , 0.0773785 ,
       0.00215426, 0.        , 0.01604311, 0.00232536, 0.01876657,
       0.06252536, 0.        , 0.02120932, 0.07127353, 0.        ,
       0.        , 0.0037282 , 0.        , 0.00523739, 0.03469318,
       0.06902182, 0.        , 0.21869814, 0.00331452, 0.22162542])

Интересовался новостями с топиками topic_3, topic_14 (что-то про политику и государство)

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [58]:
user_embeddings = {}
stats = {'mean', 'median', 'max'}
for st in stats:
    user_embeddings[st] = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, st), 1)])
    user_embeddings[st].columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings[st]['uid'] = users['uid'].values
    user_embeddings[st] = user_embeddings[st][['uid']+['topic_{}'.format(i) for i in range(25)]]
    print(user_embeddings[st].head(3))

       uid   topic_0   topic_1   topic_2  topic_3   topic_4   topic_5  \
0  u105138  0.235720  0.030263  0.000000      0.0  0.076349  0.010057   
1  u108690  0.232821  0.058638  0.010501      0.0  0.028858  0.009818   
2  u108339  0.151180  0.069225  0.000000      0.0  0.018952  0.000000   

    topic_6   topic_7   topic_8  ...  topic_15  topic_16  topic_17  topic_18  \
0  0.028544  0.123950  0.023259  ...  0.000000   0.00000       0.0  0.055424   
1  0.016648  0.000000  0.017698  ...  0.000000   0.00000       0.0  0.002468   
2  0.008024  0.056297  0.008266  ...  0.004057   0.00528       0.0  0.000000   

   topic_19  topic_20  topic_21  topic_22  topic_23  topic_24  
0  0.002646  0.016693  0.000000  0.105344  0.000000  0.111752  
1  0.000000  0.067497  0.000000  0.152888  0.000000  0.183031  
2  0.007438  0.087010  0.002217  0.216494  0.003031  0.141713  

[3 rows x 26 columns]
       uid   topic_0   topic_1   topic_2  topic_3   topic_4   topic_5  \
0  u105138  0.676995  0.181576  0.

Датасет готов - можно попробовать обучить модель. Будет три разных варианта ответов. Загрузим нашу разметку

In [59]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#import itertools
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

import matplotlib.pyplot as plt

%matplotlib inline

In [63]:
y_test = {}
preds = {}
for st in stats:
    X = pd.merge(user_embeddings[st], target, 'left')
    #разделим данные на train/test
    X_train, X_test, y_train, y_test[st] = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
    logreg = LogisticRegression()
    #обучим 
    logreg.fit(X_train, y_train)
    #наши прогнозы для тестовой выборки
    preds[st] = logreg.predict_proba(X_test)[:, 1]

Рассчитаем Precision, Recall, F_score, ROC-AUC и сохраним в отдельный датафрейм.

In [66]:
final_stats = pd.DataFrame(columns = ['precision', 'recall','f-score','roc_auc'])

In [69]:
precision = {}
recall = {}
thresholds = {}
fscore = {}
for st in stats:
    precision[st], recall[st], thresholds[st] = precision_recall_curve(y_test[st], preds[st])
    fscore[st] = (2 * precision[st] * recall[st]) / (precision[st] + recall[st])
    # locate the index of the largest f score
    ix = np.argmax(fscore[st])
    final_stats.loc[st] = [precision[st][ix], recall[st][ix], fscore[st][ix], roc_auc_score(y_test[st], preds[st])]

In [70]:
final_stats

,precision,recall,f-score,roc_auc
mean,0.535433,0.832653,0.651757,0.940631
max,0.652921,0.775510,0.708955,0.951839
median,0.670068,0.804082,0.730983,0.961472


Наиболее хорошие результаты получаются в варианте с медианой. Можно обосновать это так: вариант со средним дает хуже результаты, так как среднее больше подвержено влиянию выбросов; вариант с максимум дает хуже результаты, так как максимум плохо отражает распределение векторов пользователей в целм; а медиана - более сбалансированный и устойчивый вариант. 